<a href="https://colab.research.google.com/github/yyaroslavskiy/cuddly-chainsaw/blob/feature%2Fpollynn-del-reviews/eda/PrepareForEDA%26ProDoctorovEDA%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ast
import numpy as np

Сборка датасета

In [ ]:
df_list =[]

In [ ]:
ginekologs_1 = pd.read_csv('/content/data_ginekolog_0_2000.csv')
df_list.append(ginekologs_1)
ginekologs_2 = pd.read_csv('/content/data_ginekolog_2000_3397.csv')
df_list.append(ginekologs_2)
ginekologs_3 = pd.read_csv('/content/data_ginekolog_3397_3500.csv')
df_list.append(ginekologs_3)
ginekologs_4 = pd.read_csv('/content/data_ginekolog_3500_4500.csv')
df_list.append(ginekologs_4)
ginekologs_5 = pd.read_csv('/content/data_ginekolog_4500_5500.csv')
df_list.append(ginekologs_5)
ginekologs_6 = pd.read_csv('/content/data_ginekolog_5500_6982.csv')
df_list.append(ginekologs_6)

In [ ]:
kardiologs_1 = pd.read_csv('/content/data_kardiologs_0_1000.csv')
df_list.append(kardiologs_1)
kardiologs_2 = pd.read_csv('/content/data_kardiologs_1000_2500.csv')
df_list.append(kardiologs_2)
kardiologs_3 = pd.read_csv('/content/data_kardiologs_2500_3695.csv')
df_list.append(kardiologs_3)

In [ ]:
nevrologs_1 = pd.read_csv('/content/data_nevrologs_0 2500.csv')
df_list.append(nevrologs_1)
nevrologs_2 = pd.read_csv('/content/data_nevrologs_2500_5120.csv')
df_list.append(nevrologs_2)

In [ ]:
onkologs_1 = pd.read_csv('/content/data_onkologi_0_1395.csv')
df_list.append(onkologs_1)
onkologs_2 = pd.read_csv('/content/data_onkologi_1395_2789.csv')
df_list.append(onkologs_2)

In [ ]:
terpevts_1 = pd.read_csv('/content/data_terpevt_0_500.csv')
df_list.append(terpevts_1)
terpevts_2 = pd.read_csv('/content/data_terpevt_500_1000.csv')
df_list.append(terpevts_2)
terpevts_3 = pd.read_csv('/content/data_terpevt_1000_1500.csv')
df_list.append(terpevts_3)
terpevts_4 = pd.read_csv('/content/data_terpevt_1500_2000.csv')
df_list.append(terpevts_4)
terpevts_5 = pd.read_csv('/content/data_terpevt_2000_3000.csv')
df_list.append(terpevts_5)
terpevts_6 = pd.read_csv('/content/data_terpevt_3000_4000.csv')
df_list.append(terpevts_6)
terpevts_7 = pd.read_csv('/content/data_terpevt_4000_4500.csv')
df_list.append(terpevts_7)
terpevts_8 = pd.read_csv('/content/data_terpevt_4500_5000.csv')
df_list.append(terpevts_8)
terpevts_9 = pd.read_csv('/content/data_terpevt_5000_5500.csv')
df_list.append(terpevts_9)
terpevts_10 = pd.read_csv('/content/data_terpevt_5500_6500.csv')
df_list.append(terpevts_10)
terpevts_11 = pd.read_csv('/content/data_terpevt_6500_7500.csv')
df_list.append(terpevts_11)
terpevts_12 = pd.read_csv('/content/data_terpevt_7500_8500.csv')
df_list.append(terpevts_12)
terpevts_13 = pd.read_csv('/content/data_terpevt_8500_9500.csv')
df_list.append(terpevts_13)
terpevts_14 = pd.read_csv('/content/data_terpevt_9500_10500.csv')
df_list.append(terpevts_14)
terpevts_15 = pd.read_csv('/content/data_terpevt_10500_11090.csv')
df_list.append(terpevts_15)

In [ ]:
endokrinologs_1 = pd.read_csv('/content/data_endokrinolog_0_1274.csv')
df_list.append(endokrinologs_1)
endokrinologs_2 = pd.read_csv('/content/data_endokrinolog_1274_2549.csv')
df_list.append(endokrinologs_2)

In [ ]:
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv('prodoctorov_data1.csv')

Подготовка к EDA

In [ ]:
data = pd.read_csv('/content/prodoctorov_data1.csv')

In [ ]:
data['clinics_count'] = 0
data['clinics_parsed'] = data['clinics'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
data['clinics_count'] = data['clinics_parsed'].apply(len)

for i in range(1, 3 + 1):
    data[f'clinic_{i}_name'] = data['clinics_parsed'].apply(lambda x: x[i-1]['name'] if len(x) >= i else None)
    data[f'clinic_{i}_address'] = data['clinics_parsed'].apply(lambda x: x[i-1]['address'] if len(x) >= i else None)
    data[f'clinic_{i}_metro'] = data['clinics_parsed'].apply(lambda x: x[i-1]['metro'] if len(x) >= i else None)

data = data.drop(['clinics', 'clinics_parsed'], axis=1)

In [ ]:
data['link'] = data['link'].str.split('#').str[0]

In [ ]:
reviews_data = data[['name', 'link', 'reviews']].copy()

def parse_reviews(reviews):
    try:
        if pd.isna(reviews) or reviews == 'No value':
            return []
        return ast.literal_eval(reviews)
    except:
        return []

reviews_data['reviews_parsed'] = reviews_data['reviews'].apply(parse_reviews)
reviews_data = reviews_data[reviews_data['reviews_parsed'].apply(len) > 0]
reviews_exploded = reviews_data.explode('reviews_parsed')

reviews_data = pd.DataFrame({
    'doctor_name': reviews_exploded['name'],
    'doctor_link': reviews_exploded['link'],
    'rate': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('rate')),
    'date': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('date')),
    'comment': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('comment')),
    'clinic': reviews_exploded['reviews_parsed'].apply(lambda x: x.get('clinic'))
})

In [ ]:
reviews_data = reviews_data.reset_index(drop=True)

In [ ]:
del data['reviews']

In [ ]:
data = data.replace('No value', np.nan)

to_str_col = ['name', 'link', 'speciality', 'clinic_1_name', 'clinic_2_name', 'clinic_3_name', 'clinic_1_address', 'clinic_2_address', 'clinic_3_address']
for col in to_str_col:
  data[col] = data[col].astype('string')

to_cat_col = ['clinic_1_metro', 'clinic_2_metro', 'clinic_3_metro']
for col in to_cat_col:
  data[col] = data[col].astype('category')

data['rating'] = data['rating'].astype('float64')

data['price'] = data['price'].astype('float64')

to_num_col = ['price', 'experience', 'review_count','clinics_count']
for col in to_num_col:
  data[col] = data[col].astype('Int64')


Доктора

In [ ]:
data = data.drop_duplicates(subset=['link'])


In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data = data[data['experience']<300.0]

In [ ]:
data = data[(data['price'].notna())|(data['experience'].notna())|(data['rating'].notna())|(data['review_count']!= 0)]

In [ ]:
data.describe()

In [ ]:
data.info()

ОТЗЫВЫ

In [ ]:
reviews_data = reviews_data.drop_duplicates()

In [ ]:
reviews_data = reviews_data.replace('No value', np.nan)

to_str_col = ['doctor_name', 'doctor_link','comment','clinic']
for col in to_str_col:
  reviews_data[col] = reviews_data[col].astype('string')

reviews_data['rate'] = pd.to_numeric(reviews_data['rate'], errors='coerce')

reviews_data['date'] = pd.to_datetime(reviews_data['date'], format='%Y-%m-%d')


In [ ]:
reviews_data['comment'] = reviews_data['comment'].str.replace('\u200b', '')

In [ ]:
reviews_data.info()

In [ ]:
reviews_data['rate'].value_counts().head(10)

In [ ]:
reviews_data['date'].value_counts().head(10)

In [ ]:
reviews_data = reviews_data[(reviews_data['rate'].notna())|(reviews_data['comment'].notna())]

In [ ]:
reviews_data.info()

In [ ]:
reviews_data.describe()

In [ ]:
data.to_csv('data_prodoctorov_to_eda.csv')

In [ ]:
reviews_data.to_csv('data_prodoctorov_reviews_to_eda.csv')